In [46]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import os
import sys
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names
import matplotlib.pyplot as plt

In [47]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

current_dir = os.getcwd()
print(current_dir)

plots_path = os.path.join(current_dir, 'plots') + "/"
tables_path = os.path.join(current_dir, 'tables') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
if not os.path.exists(plots_path):
    os.makedirs(plots_path)
if not os.path.exists(tables_path):
    os.makedirs(tables_path)
if not os.path.exists(latex_path):
    os.makedirs(latex_path)


cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [48]:
# get unique generator names
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [49]:
cases_resources_capacities_table = get_printable_resource_names(cases_resources_capacities)
# unique_resources_printable = get_printable_resource_names(unique_resources)

In [50]:
# Get the columns that start with 'BESS'
bess_columns = [col for col in cases_resources_capacities_table.columns if col.startswith('BESS')]

# Create a temporary vector to hold the maximum values of the BESS columns
bess_capacities = cases_resources_capacities_table[bess_columns].max(axis=1)

# Drop the original BESS columns
cases_resources_capacities_table_all = cases_resources_capacities_table.drop(columns=bess_columns)

# add the maximum BESS capacities to the table
cases_resources_capacities_table_all['BESS'] = bess_capacities



In [51]:
# Round the values in the table to zero decimal places and convert to integers
cases_resources_capacities_table_rounded = cases_resources_capacities_table_all.copy()
columns_to_round = [col for col in cases_resources_capacities_table_rounded.columns if col != 'Case Name']
cases_resources_capacities_table_rounded[columns_to_round] = cases_resources_capacities_table_rounded[columns_to_round].round(0).astype(int)

In [52]:
# save the table to a csv file in tables_path, and latex format in latex_path
cases_resources_capacities_table_rounded.to_csv(os.path.join(tables_path, 'table2_capacity_in_gw.csv'), index=False)
cases_resources_capacities_table_rounded.to_latex(os.path.join(latex_path, 'table2_capacity_in_gw.tex'), index=False, escape=False)